Code written in Oct 2021 by Yuhan Wang
only suitable for UFMs when TESes are in normal stage
instead of fitting to noise model, this takes median noise from 5Hz to 50Hz
different noise levels here are based on phase 2 noise target and noise model after considering johnson noise at 100mK

# Imports/Setup
Not all imports are required, some are here for convince.

In [ ]:
# selects a non-interactive plotting backend
import matplotlib
matplotlib.use('Agg')

# required imports and favorite packages.
import pysmurf.client
import argparse
import numpy as np
import os
import time
import glob
from sodetlib.det_config import DetConfig
import numpy as np
from scipy.interpolate import interp1d
import argparse
import time
import csv
import scipy.signal as signal
import matplotlib.pyplot as plt


# LoadS - Getting control of the SMuRF blade by slot number
This is a class that can load and operate on an abstract number of SMuRF controllers
obtained by slot number.

In [ ]:
class LoadS:
    """
    For obtaining and initializing and abstract number of SMuRF controllers (identified by slot number)
    for testing, commonly this is denoted as python objected denoted as an uppercase 'S'.

    For a single S, and cfg use:

        slot_num = 1
        load_s = LoadS(slot_nums={slot_num}, verbose=verbose)
        cfg = load_s.S_dict[slot_num]
        S = load_s.S_dict[slot_num]

    For multiple S and cfg values use:

        slot_nums = [2, 0, 4]  # just showing order is not important.
        load_s = LoadS(slot_nums=slot_nums, verbose=verbose)
        for slot_num in sorted(load_s.slot_nums):
            cfg = load_s.S_dict[slot_num]
            S = load_s.S_dict[slot_num]
    """
    def __init__(self, slot_nums=None, verbose=True):
        """
        Proved with a slot numbers, an instance of this class will automatically load and configure
        each SMuRF slot. The smurf slots are accessible through the instance variables self.cfg_dict,
        self.S_dict, and self.logs_dict. As the names of these variables suggest, each variable is a
        dictionary with the values of the slot numbers as keys to S, cfg, log, for a single SMuRF slot.

        :param slot_nums: single value or iterable of slot number (or numbers) to load and control.
        :param verbose: bool - Toggles print() statements for the actions within the methods methods of this
                               class
        """
        # record the information used across the methods in this class
        self.verbose = verbose
        if slot_nums is None:
            self.slot_nums = set()
        else:
            self.slot_nums = {int(slot_num) for slot_num in slot_nums}

        # initialize the controller storage variables
        self.cfg_dict = {}
        self.S_dict = {}
        self.logs_dict = {}

        # load the controllers here if
        for slot_num in sorted(self.slot_nums):
            self.load_single_slot(slot_num=slot_num)

    def load_single_slot(self, slot_num):
        """
        This loads the controller for a single SMuRF slot:

        This method populates the instance variables of self.cfg_dict, self.S_dict, and self.logs_dict
        using 'slot_num' as the key.

        :param slot_num: int - The SMuRF slot number.
        """
        slot_num = int(slot_num)
        # this may already by in self.slot_nums, but we add it here to capture slot_nums specified outside of __init__
        self.slot_nums.add(slot_num)
        if self.verbose:
            print(f'Getting SMuRF control for slot: {slot_num}')
        # get control
        self.cfg_dict[slot_num] = DetConfig()
        # load config files
        self.cfg_dict[slot_num].load_config_files(slot=slot_num)
        if self.verbose:
            print(DetConfig)
        # get the controller with the configuration
        self.S_dict[slot_num] = self.cfg_dict[slot_num].get_smurf_control(make_logfile=True)
        if self.verbose:
            print(f'  log file found at {self.S_dict[slot_num].logfile}')
            print(f'  plotting directory at {self.S_dict[slot_num].plotdir}')
        # automatically configure the controllers after acquiring them
        self.configure_single_slot(slot_num=slot_num)

    def configure_single_slot(self, slot_num, set_downsample_factor=20):
        """

        :param slot_num: int - The SMuRF slot number.
        :param set_downsample_factor: float - The SMuRF downsampling factor.
        """
        self.S_dict[slot_num].all_off()
        self.S_dict[slot_num].set_rtm_arb_waveform_enable(0)
        self.S_dict[slot_num].set_filter_disable(0)
        self.S_dict[slot_num].set_downsample_factor(set_downsample_factor)
        self.S_dict[slot_num].set_mode_dc()

# SingleBand - Set up a single band
There are a lot of ways to set up or configure a single band (bias line). This class stores
the fundamental methods for setting up a given band. Some methods in this class script these
fundamental methods. The methods at the end of this class are more fundamental while the methods
at the beginning are more complex scrips.

In [ ]:
class SingleBand:
    """
    SingleBand is a class to operate on a single band (bias line) for a given SMuRF controller.
    """
    def __init__(self, S, cfg, band, auto_startup=False, verbose=True):
        """
        :param S: SMuRF controller object.
        :param cfg: SMuRF configuration object
        :param band: int - band identifier (bias line)
        :param auto_startup: bool - default is False. When True, the method .startup() runs,
                                    setting up the band in a standard script.
        :param verbose: bool - default is True: When True, toggles print() statements for the
                               various actions taken by the methods in this class. False,
                               runs silently.
        """
        self.S = S
        self.cfg = cfg
        self.band = band
        self.verbose = verbose
        if auto_startup:
            self.startup()

    def startup(self):
        """
        A standard (default) setup for a single band (bias line)
        """
        if self.verbose:
            print(f'setting up band {self.band}')
        self.set_att_dc()
        self.set_tone_power()
        self.est_phase_delay()
        self.set_synthesis_scale()
        self.find_freq()
        self.run_serial_gradient_descent()
        self.run_tracking_setup()

    def set_att_dc(self):
        self.S.set_att_dc(band, cfg.dev.bands[band]["dc_att"])
        if self.verbose:
            print(f'band {self.band} dc_att {self.S.get_att_dc(band)}')

    def set_tone_power(self):
        self.S.amplitude_scale[band] = cfg.dev.bands[band]["drive"]
        if self.verbose:
            print(f"band {band} tone power {S.amplitude_scale[band]}")

    def est_phase_delay(self):
        if self.verbose:
            print('estimating phase delay...')
        self.S.estimate_phase_delay(band)

    def set_synthesis_scale(self):
        if self.verbose:
            print("setting synthesis scale...")
        # hard coding it for the current fw
        S.set_synthesis_scale(band, 1)

    def find_freq(self):
        if self.verbose:
            print("running S.find_freq...")

        self.S.find_freq(band, tone_power=self.cfg.dev.bands[self.band]["drive"], make_plot=True)

    def setup_notches(self):
        if self.verbose:
            print("running setup notches")
        self.S.setup_notches(self.band, tone_power=self.cfg.dev.bands[self.band]["drive"], new_master_assignment=True)

    def run_serial_gradient_descent(self):
        if self.verbose:
            print("running serial gradient descent and eta scan")
            S.run_serial_gradient_descent(band)
            S.run_serial_eta_scan(band)

    def run_tracking_setup(self):
        if self.verbose:
            print("running tracking setup...")
        self.S.set_feedback_enable(self.band, 1)
        self.S.tracking_setup(
            self.band,
            reset_rate_khz=self.cfg.dev.bands[self.band]["flux_ramp_rate_khz"],
            fraction_full_scale=self.cfg.dev.bands[self.band]["frac_pp"],
            make_plot=False,
            save_plot=False,
            show_plot=False,
            channel=self.S.which_on(self.band),
            nsamp=2 ** 18,
            lms_freq_hz=None,
            meas_lms_freq=True,
            feedback_start_frac=self.cfg.dev.bands[self.band]["feedback_start_frac"],
            feedback_end_frac=self.cfg.dev.bands[self.band]["feedback_end_frac"],
            lms_gain=self.cfg.dev.bands[self.band]["lms_gain"])
        if self.verbose:
            print("  tracking setup complete")

    def check_lock(self):
        if self.verbose:
            print('checking tacking lock...')
        self.S.check_lock(self.band,
            reset_rate_khz=self.cfg.dev.bands[self.band]["flux_ramp_rate_khz"],
            fraction_full_scale=self.cfg.dev.bands[self.band]["frac_pp"],
            lms_freq_hz=None,
            feedback_start_frac=self.cfg.dev.bands[self.band]["feedback_start_frac"],
            feedback_end_frac=self.cfg.dev.bands[self.band]["feedback_end_frac"],
            lms_gain=self.cfg.dev.bands[self.band]["lms_gain"])
        if self.verbose:
            print('  tracking lock check complete.')

# TimeStreamData
### A class for acquiring and viewing time stream data.

Each instance of this class can acquire time streams, however each instance is only
used to view a single time stream at a time. For viewing multiple time streams,
initiate multiple instances of this class.

In [ ]:
class TimeStreamData:
    """
    View and acquire time steam data from a single SMuRF controller object. This class
    is designed to be an acquisition tool and quick look only. More advance analysis
    may be available elsewhere.

    This can be used to view existing time stream files. See __init__() and read_ts() methods.

    Note: to load and view multiple time streams simultaneously use multiple instances
    of this class. This class only holds one time stream of data at a time. Reading
    additional time stream data will overwrite previously loaded time stream data.

    """

    def __init__(self, S, cfg, nperseg=2**18, data_path=None, verbose=True):
        """
        :param S: SMuRF controller object.
        :param cfg: SMuRF configuration object
        :param nperseg: nperseg int - default is 2**18 – The number of samples used in the
                                      PSD estimator. See scipy.signal.welch.
        :param data_path: path (str) - default is None. When None, auto-loading of data from the
                                       __init__() is disabled. When a path sting is given, the
                                       method is read_ts() is called and time stream file located
                                       at path is read in and analyzed.
        :param verbose: bool - default is True: When True, toggles print() statements for the
                               various actions taken by the methods in this class. False,
                               runs silently.
        """
        self.S = S
        self.cfg = cfg
        self.verbose = verbose

        self.nperseg = nperseg
        self.detrend = 'constant'

        self.data_path = data_path
        self.start_time = None
        self.fs = None
        self.bands, self.channels = None, None
        self.phase, self.mask, self.tes_bias = None, None, None
        self.sample_nums = None
        self.t_array = None
        self.stream_by_band_by_channel = None

        # in method get_median_bias_wl()
        self.wl_list = None
        self.wl_median = None

        if self.data_path is not None:
            self.read_ts()

    def take_ts(self, stream_time=20):
        """
        Aquire time stream data.

        :param stream_time: float - the sleep time in seconds to wait while acquiring time stream data.
        """
        self.start_time = self.S.get_timestamp()

        # non blocking statement to start time stream and return the dat filename
        try:
            self.data_path = self.S.stream_data_on()
            # collect stream data
            if self.verbose:
                print(f'Sleeping for {stream_time} seconds to take time stream data...\n')
            time.sleep(stream_time)
            # end the time stream
        except:
            self.S.stream_data_off()
            print(f'Stream Off command sent')
            raise
        else:
            self.S.stream_data_off()
            if self.verbose:
                print(f'Stream Off command sent')
        # read the data that was taken
        self.read_ts()

    def read_ts(self, data_path=None):
        """
        Collect Data and transform it to a shape for plotting and analysis

        :param data_path: path (str) - default is None. When None, the path from the __init__()
                                       method at self.data_path is used. When this is a path like
                                       str, the instance variable self.data_path is overwritten
                                       and the time stream data at data_path is loaded.
        """
        if data_path is not None:
            self.data_path = data_path
        # Read the data
        timestamp, self.phase, self.mask, self.tes_bias = self.S.read_stream_data(self.data_path, return_tes_bias=True)
        self.fs = self.S.get_sample_frequency()
        dirname, basename = os.path.split(self.data_path)
        file_handle, file_extention = basename.rsplit('.', 1)
        self.start_time = file_handle
        print(f'loaded the .dat file at: {self.data_path}')

        # hard coded variables
        self.bands, self.channels = np.where(self.mask != -1)
        self.phase *= self.S.pA_per_phi0 / (2.0 * np.pi)  # uA
        self.sample_nums = np.arange(len(self.phase[0]))
        self.t_array = self.sample_nums / self.fs

        # reorganize the data by band then channel
        self.stream_by_band_by_channel = {}
        for band, channel in zip(self.bands, self.channels):
            if band not in self.stream_by_band_by_channel.keys():
                self.stream_by_band_by_channel[band] = {}
            ch_idx = self.mask[band, channel]
            self.stream_by_band_by_channel[band][channel] = self.phase[ch_idx]

    def get_median_bias_wl(self, fmin=float('-int'), fmax=(float('inf'))):
        """
        Do some per-band analysis on the time stream data.

        Populate the per-band dictionaries of self.wl_list (self.wl_list[band] is a list)
        and self.wl_median (self.wl_median[band] is a float).

        :param fmin: float - default is float('-inf'). The minimum frequency to consider,
                             frequencies below this value will be masked during calculations.
        :param fmax: float - default is float('inf'). The maximum frequency to consider,
                             frequencies above this value will be masked during calculations.
        """
        self.wl_list = {}
        self.wl_median = {}
        for band in sorted(self.stream_by_band_by_channel.keys()):
            self.wl_list[band] = []
            channels_this_band = self.stream_by_band_by_channel[band]
            for channel in sorted(channels_this_band.keys()):
                stream_single_channel = channels_this_band[channel]
                f, Pxx = signal.welch(stream_single_channel, nperseg=self.nperseg, fs=self.fs, detrend=self.detrend)
                Pxx = np.sqrt(Pxx)
                fmask = (fmin < f) & (f < fmax)

                wl = np.median(Pxx[fmask])
                self.wl_list[band].append(wl)
            self.wl_median = np.median(self.wl_list[band])

    def plot_ts(self):
        """
        Plot the time stream data.

        No user options, only hacking these values...
        """
        # # Plot layout and other defaults
        # Guide lines and hard coded plot elements
        psd_guild_lines_hz = [1.4, 60.0]
        psd_guild_line_colors = ['darkorchid', 'firebrick']
        psd_guild_line_alpha = 0.2
        psd_y_min_pa_roothz = 1.0
        psd_y_max_pa_roothz = 1.0e4

        # figure margins in figure coordinates
        frame_on = False
        left = 0.08
        bottom = 0.02
        right = 0.99
        top = 0.98
        figure_width_inches = 12
        figure_height_inches = 24
        inter_band_spacing_x = 0.05
        inter_band_spacing_y = 0.05
        phase_to_psd_height_ratio = 0.5
        between_phase_and_psd_spacing_y = 0.02
        # basic axis layout choices
        columns = 2
        # layout calculations
        sorted_bands = sorted(self.stream_by_band_by_channel.keys())
        num_of_bands = len(sorted_bands)
        rows = int(np.ceil(num_of_bands / float(columns)))
        single_band_width = (right - left - ((columns - 1) * inter_band_spacing_x)) / float(columns)
        single_band_height = (top - bottom - ((rows - 1) * inter_band_spacing_y)) / float(rows)
        available_band_height = single_band_height - between_phase_and_psd_spacing_y
        single_phase_height = available_band_height * phase_to_psd_height_ratio / (phase_to_psd_height_ratio + 1.0)
        single_psd_height = available_band_height - single_phase_height

        # figure and axis-handle setup
        fig = plt.figure(figsize=(figure_width_inches, figure_height_inches))
        ax_dict_phase = {}
        ax_dict_psd = {}
        left_ax_coord = left
        top_ax_phase_coord = top
        for counter, band in list(enumerate(sorted_bands)):
            # local calculations
            bottom_ax_phase_coord = top_ax_phase_coord - single_phase_height
            bottom_ax_psd_coord = bottom_ax_phase_coord - between_phase_and_psd_spacing_y - single_psd_height
            phase_coords = [left_ax_coord, bottom_ax_phase_coord, single_band_width, single_phase_height]
            psd_coords = [left_ax_coord, bottom_ax_psd_coord, single_band_width, single_psd_height]
            # create the axis handles
            ax_dict_phase[band] = fig.add_axes(phase_coords, frameon=frame_on)
            ax_dict_psd[band] = fig.add_axes(psd_coords, frameon=frame_on)
            # reset things for the next loop
            if ((counter + 1) % columns) == 0:
                # case where the next axis is on a new row
                left_ax_coord = left
                top_ax_phase_coord -= single_band_height + inter_band_spacing_y
            else:
                # case where the next axis is in to next column
                left_ax_coord += single_band_width + inter_band_spacing_x

        # plot the band channel data
        for counter2, band in list(enumerate(sorted_bands)):
            stream_single_band = self.stream_by_band_by_channel[band]
            ax_phase_this_band = ax_dict_phase[band]
            ax_psd_this_band = ax_dict_psd[band]
            for channel in sorted(stream_single_band.keys()):
                # phase
                stream_single_channel = stream_single_band[channel]
                stream_single_channel_norm = stream_single_channel - np.mean(stream_single_channel)
                ax_phase_this_band.plot(self.t_array, stream_single_channel_norm, color='C0', alpha=0.002)
                # psd
                f, Pxx = signal.welch(stream_single_channel, nperseg=self.nperseg, fs=self.fs,
                                      detrend=self.detrend)
                Pxx = np.sqrt(Pxx)
                ax_psd_this_band.loglog(f, Pxx, color='C0', alpha=0.002)

            # phase
            band_yield = len(stream_single_band)
            ax_phase_this_band.set_xlabel('time [s]')
            if counter2 % columns == 0:
                ax_phase_this_band.set_ylabel('Phase [pA]')
            ax_phase_this_band.grid()
            ax_phase_this_band.set_title(f'band {band} yield {band_yield}')
            ax_phase_this_band.set_ylim([-10000, 10000])
            # psd
            for line_hz, line_color in zip(psd_guild_lines_hz, psd_guild_line_colors):
                # add the guild lines to the plots
                ax_psd_this_band.plot([line_hz, line_hz],
                                      [psd_y_min_pa_roothz, psd_y_max_pa_roothz],
                                      color=line_color, alpha=psd_guild_line_alpha,
                                      ls='dashed')
                ax_psd_this_band.text(x=line_hz, y=psd_y_max_pa_roothz, s=f"{line_hz} Hz", color=line_color,
                                      rotation=315, alpha=0.6,
                                      ha='left', va='top')
            ax_psd_this_band.set_xlabel('Frequency [Hz]')
            if counter2 % columns == 0:
                ax_psd_this_band.set_ylabel('Amp [pA/rtHz]')
            ax_psd_this_band.grid()
            ax_psd_this_band.set_ylim([psd_y_min_pa_roothz, psd_y_max_pa_roothz])

        save_name = f'{self.start_time}_band_noise_stack.png'
        print(f'Saving plot to {os.path.join(S.plot_dir, save_name)}')
        plt.savefig(os.path.join(S.plot_dir, save_name))
        plt.close(fig=fig)

In [ ]:
class  GroupedBiases:
    def __init__(self, S, cfg, bias_group, verbose=True):
        """
        :param S: SMuRF controller object.
        :param cfg: SMuRF configuration object
        :param bias_group: set - Needs to be some iterable that can be turned into a set of bands (bias lines).
                                 All the bands in this set will be operated on.
        :param verbose: bool - default is True: When True, toggles print() statements for the
                               various actions taken by the methods in this class. False,
                               runs silently.
        """
        self.S = S
        self.cfg = cfg
        self.verbose = verbose
        self.bias_group = set(bias_group)
        self.bias_group_list = sorted(self.bias_group)

    def overbias_tes(self, sleep_time=120):
        if self.verbose:
            print("Overbiasing TES")
        self.S.overbias_tes_all(bias_groups=self.bias_group_list,
                                overbias_wait=1, tes_bias=12, cool_wait=3, high_current_mode=False,
                                overbias_voltage=12)
        if self.verbose:
            print("waiting for thermal environment get stabilized, sleeping {sleep_time} seconds.")
        time.sleep(sleep_time)

In [ ]:
class AutoTune:
    """
    A class to tune parameters independently across bias lines, checking the white noise levels
    and making mappings with parameter values. These mappings are used to inform subsequent
    tunings.

    This class makes heavy use of the TimeStreamData class. An instance of that class is 
    started in the __init__() method of this class, see self.time_stream_data.

    This is a bit experimental. There are many operational topologies for optimizing detectors,
    we will choose one that accounts for operational constraints of real devices.
    """
    def __init__(self, S, cfg, bias_group, nperseg=2**12, verbose=True):
        """
        :param S: SMuRF controller object.
        :param cfg: SMuRF configuration object
        :param bias_group: set - Needs to be some iterable that can be turned into a set of bands (bias lines).
                         All the bands in this set will be operated on.
        :param nperseg: nperseg int - default is 2**18 – The number of samples used in the
                                      PSD estimator. See scipy.signal.welch.
        :param verbose: bool - default is True: When True, toggles print() statements for the
                               various actions taken by the methods in this class. False,
                               runs silently.
        """
        # user defined variables and __init__ declared variables. 
        self.S = S
        self.cfg = cfg
        self.verbose = verbose
        self.bias_group = set(bias_group)
        self.bias_group_list = sorted(self.bias_group)
        self.nperseg = nperseg

        # in method tuner()
        self.wl_medians = None
        self.noise_floors = None
        self.uc_atten_wl_median_tune_map_per_band = {}
        self.uc_atten_best_per_band = None
        self.lowest_wl_index_per_band = None
        self.wl_median_per_band = None

        self.time_stream_data = TimeStreamData(S=S, cfg=cfg, nperseg=self.nperseg, verbose=verbose)

    def single_time_stream(self, stream_time=120, do_plot=False, fmin=5, fmax=50):
        """
        Get a single time stream and use the TimeStreamData class to do some additional processing.
        
        :param stream_time: float - The default is 120 seconds. The sleep time in seconds to wait 
                                    while acquiring time stream data.
        :param do_plot: bool - default is False. When True, a standard diagnostic plot is rendered
                               and saved for this time stream.
        :param fmin: float - default is float('-inf'). The minimum frequency to consider,
                             frequencies below this value will be masked during calculations.
        :param fmax: float - default is float('inf'). The maximum frequency to consider,
                             frequencies above this value will be masked during calculations.                   
        """
        self.time_stream_data.take_ts(stream_time=stream_time)
        if do_plot:
            self.time_stream_data.plot_ts()
        self.time_stream_data.get_median_bias_wl(fmin=fmin, fmax=fmax)
        if self.verbose:
            print(f'wl_median {self.time_stream_data.wl_median}')


    def tuner_up_atten(self, uc_attens_centers_per_band, tune_type_per_band=None, stream_time=120,
                       do_plots=False, fmin=5, fmax=50):
        """
        Acquire multiple time streams to test up conversation attenuation settings values for all bias lines.
        
        : param uc_attens_centers_per_band: dict - required. Expecting a dictionary with keys that
                                                   are bands (bias lines). The values for each key 
                                                   should be up conversation attenuation setting 
                                                   values, int. For bands where 
                                                   tune_type_per_band[band] == 'single' then 
                                                   uc_attens_centers_per_band[band] is the only 
                                                   value measured this method. If 
                                                   tune_type_per_band[band] in {'fine', 'rough'}, 
                                                   then the value of uc_attens_centers_per_band[band]
                                                   is the center value of a spread of a searches 
                                                   over several time streams.
        :param tune_type_per_band: dict - default is None. Expecting a dictionary with keys that
                                          are bands (bias lines) and at least the all the keys
                                          also in uc_attens_centers_per_band.keys(). The values 
                                          for each key should be the strings 'single' or 'fine', 
                                          all other values trigger a 'rough' tuning. A single tuning 
                                          does only one tuning a the up conversation attenuation
                                          at the value of uc_attens_centers_per_band[band].
                                          'fine' does a tuning of near by uc_atten values, while
                                          'rough' or the default tuning is using uc_atten values
                                          with a larger spread. None, the default, will set
                                          tune_type_per_band[band] = 'single' for every band in 
                                          uc_attens_centers_per_band.keys()
        :param stream_time: float - The default is 120 seconds. Applies to all tunings started in
                                    this method.The sleep time in seconds to wait while acquiring 
                                    time stream data.
        :param do_plots: bool - default is False. Applies to all tunings started in this method.
                                When True, a standard diagnostic plot is rendered and saved for each
                                time stream in this method. 
        :param fmin: float - default is float('-inf'). Applies to all tunings started in this method.
                             The minimum frequency to consider, frequencies below this value will be 
                             masked during calculations.
        :param fmax: float - default is float('inf'). Applies to all tunings started in this method.
                             The maximum frequency to consider, frequencies above this value will 
                             be masked during calculations. 
        """
        if tune_type_per_band is None:
            tune_type_per_band = {band: 'single' for band in uc_attens_centers_per_band.keys()}
        # determine the settings to test
        uc_attens_per_band = {}
        for band in uc_attens_centers_per_band:
            uc_attens_per_band[band] = []
            uc_attens_this_band = uc_attens_per_band[band]
            uc_atten_center_this_band = uc_attens_centers_per_band[band]
            uc_atten_wl_median_tune_map_this_band = self.uc_atten_wl_median_tune_map_per_band[band]
            tune_type_this_band = tune_type_per_band[band]
            tune_type_this_band = tune_type_this_band.lower()
            if tune_type_this_band is 'fine':
                diffs_from_center = [-2, -1, 0, 1, 2]
            elif tune_type_this_band is 'single':
                diffs_from_center = [0]
            else:
                diffs_from_center = [-10, -5, 0, 5, 10]

            for diff_from_center in diffs_from_center:
                test_setting = uc_atten_center_this_band + diff_from_center
                # try to use settings that have not yet been tested on.
                if tune_type_this_band != 'fine' and \
                        test_setting in uc_atten_wl_median_tune_map_this_band.keys():
                    # add or subtract a value to not repeat a measurements.
                    if diff_from_center < 0:
                        small_perturbations = [-1, 1, -2, 2]
                    else:
                        small_perturbations = [1, -1, 2, -2]
                    for small_perturbation in small_perturbations:
                        test_setting2 = test_setting + small_perturbation
                        if test_setting2 not in uc_atten_wl_median_tune_map_this_band.keys():
                            test_setting = test_setting2
                            break
                uc_attens_this_band.append(test_setting)
        # test for results
        bands = sorted(uc_attens_per_band.keys())
        bands_remaining = set(bands)
        self.wl_medians = {band: [] for band in bands}
        self.noise_floors = {band: [] for band in bands}
        index_counter = 0
        while bands_remaining != set():
            for band in sorted(bands_remaining):
                try:
                    uc_atten = uc_attens_per_band[band][index_counter]
                except IndexError:
                    bands_remaining.remove(band)
                else:
                    self.S.set_att_uc(band, uc_atten)
                    S.tracking_setup(band,
                                     reset_rate_khz=cfg.dev.bands[band]["flux_ramp_rate_khz"],
                                     fraction_full_scale=cfg.dev.bands[band]["frac_pp"],
                                     make_plot=False,
                                     save_plot=False,
                                     show_plot=False,
                                     channel=S.which_on(band),
                                     nsamp=2 ** 18,
                                     lms_freq_hz=None,
                                     meas_lms_freq=True,
                                     feedback_start_frac=cfg.dev.bands[band]["feedback_start_frac"],
                                     feedback_end_frac=cfg.dev.bands[band]["feedback_end_frac"],
                                     lms_gain=cfg.dev.bands[band]["lms_gain"])

            index_counter += 1
            # record the data
            self.single_time_stream(stream_time=stream_time, do_plot=do_plots, fmin=fmin, fmax=fmax)
            # collect the data after the time streams
            for band in bands:
                self.wl_medians[band].append(self.time_stream_data.wl_median[band])
                self.noise_floors[band].append(self.time_stream_data.wl_list[band])
        # analyze the results and set the optimal values.
        for band in bands:
            wl_medians_this_band = self.wl_medians[band]
            uc_attens__this_band = uc_attens_per_band[band]
            # print results once per band
            lowest_wl_index = wl_medians_this_band.index(min(wl_medians_this_band))
            wl_median = self.wl_medians[band][lowest_wl_index]
            uc_atten_best_this_band = uc_attens__this_band[lowest_wl_index]
            self.uc_atten_best_per_band[band] = uc_atten_best_this_band
            self.lowest_wl_index_per_band[band] = lowest_wl_index
            self.wl_median_per_band[band] = wl_median
            if self.verbose:
                print(f'T est results for "up conversion attenuation" band:{band} tune_type:{tune_type_per_band[band]}')
                print(f'  medians per up-conversation attenuation index:\n  {self.wl_medians[band]}')
                print(f'  {uc_atten_best_this_band} up-conversation attenuation index with the lowest median ' +
                      f'channel noise {wl_median}')
                channel_length = self.time_stream_data.stream_by_band_by_channel[band]
                print(f"  lowest WL: {wl_median} with {channel_length} channels")
            # set this tuning
            self.S.set_att_uc(band, uc_atten_best_this_band)
            # record the up conversation attenuation with noise level map
            self.uc_atten_wl_median_tune_map_per_band[band] = {uc_atten: wl_median for uc_atten, wl_median
                                                               in zip(uc_attens__this_band, wl_medians_this_band)}

    def tune_selector_up_atten(self, uc_attens_centers_per_band=None, loop_count_max=5,
                               stream_time=120, do_plots=False, fmin=5, fmax=50):
        """
        :param uc_attens_centers_per_band: dict - default is None. Expecting a dictionary with keys that
                                                  are bands (bias lines). The values for each key 
                                                  should be up conversation attenuation setting 
                                                  values, int. This sets the intial test value of
                                                  uc_atten for each band. None set all bands to 
                                                  a uc_atten values that is a constant. Ideally, 
                                                  this would read ina record best of value.
        :param loop_count_max: int - default is 5. The number of loops to attempt tuning before
                                     a failed loop exit. 
        :param stream_time: float - The default is 120 seconds. Applies to all tunings started in
                                    this method.The sleep time in seconds to wait while acquiring 
                                    time stream data.
        :param do_plots: bool - default is False. Applies to all tunings started in this method.
                                When True, a standard diagnostic plot is rendered and saved for each
                                time stream in this method. 
        :param fmin: float - default is float('-inf'). Applies to all tunings started in this method.
                             The minimum frequency to consider, frequencies below this value will be 
                             masked during calculations.
        :param fmax: float - default is float('inf'). Applies to all tunings started in this method.
                             The maximum frequency to consider, frequencies above this value will 
                             be masked during calculations. 
        """
        # do an initial of the up conversation attenuators
        if uc_attens_centers_per_band is None:
            uc_attens_centers_per_band = {band: 12 for band in self.bias_group}
        self.tuner_up_atten(uc_attens_centers_per_band=uc_attens_centers_per_band, tune_type_per_band=None,
                            stream_time=stream_time, do_plots=do_plots, fmin=fmin, fmax=fmax)

        # check the tuning and do some optimization
        bias_bands_to_tune = set(uc_attens_centers_per_band.keys())
        loop_count = 0
        while bias_bands_to_tune == set():
            # break from the loop to indicate a non ideal exit
            if loop_count < loop_count_max:
                print(f'loop count: {loop_count}, is at it maximum allowed value.')
                break
            loop_count += 1
            # # the tuning selection process
            # determine the best tuning in the global map
            uc_attens_centers_per_band = {}
            tune_type_per_band = {}
            for band in sorted(bias_bands_to_tune):
                uc_atten_wl_median_this_band = self.uc_atten_wl_median_tune_map_per_band[band]
                # return the key (up converstion attetnuation value) of the lowest value in an array
                uc_atten_min_wl_median = min(uc_atten_wl_median_this_band, key=uc_atten_wl_median_this_band.get)
                wl_median_this_band = self.uc_atten_wl_median_tune_map_per_band[band][uc_atten_min_wl_median]
                uc_attens_centers_per_band[band] = uc_atten_min_wl_median
                # rough and fine tuning need to be done per band.
                print(f"WL: {wl_median_this_band} for band {band}")
                if 150 < wl_median_this_band:
                    raise ValueError(f"WL: {wl_median_this_band} is greater then 150. Something might be wrong,\n" +
                                     f"Up and/or Down conversation attenuation may not be correctly set\nPlease investigate")
                elif 60 < wl_median_this_band <= 150:
                    print(f"  Attempting rough tuning for band:{band}")
                    tune_type_per_band[band] = 'rough'

                # apply the tuning
                self.tuner_up_atten(uc_attens_centers_per_band=uc_attens_centers_per_band,
                                    tune_type_per_band=tune_type_per_band,
                                    stream_time=stream_time, do_plots=do_plots, fmin=fmin, fmax=fmax)


# The Script Example
All the classes and import statements above only set the stage, this where the dance begins!

## Example Parameters
These are parameters chosen to emulate the functionality of ufm_optimize_quick_normal.py.
However, several parameters have been abstracted in the classes their methods.
See the doc-strings in the various classes for more information about use cases.

In [ ]:
# example parameters
band = 4
slot_num = 2
bias_group = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
stream_time = 20
nperseg = 2**12
verbose = True

## Load the SMuRF controller
The load_s class instance can support multiple controllers, accessible by slot number in the
dictionaries load_s.cfg_dict, load_s.S_dict, and load_s.log_dict.

Initialization is automatic, see the doc-strings in the 'LoadS' class for details.

In [ ]:
# load a single S, or SMuRF controller instance for a given slot number
load_s = LoadS(slot_nums=[slot_num], verbose=verbose)
cfg = load_s.cfg_dict[slot_num]
S = load_s.S_dict[slot_num]

## Lock and Configure on a Single Band
Setup for locking on a single `band`. When auto_startup is True, the method SingleBand.startup()
handles all the standard configuration settings. SingleBand.startup() is really a script and can
have components added and deleted.

In [ ]:
# configure a single band
single_band = SingleBand(S=S, cfg=cfg, band=band, auto_startup=True, verbose=verbose)
single_band.check_lock()

## Overbiases the detectors, an example of GroupedBiases
The GroupedBiases is a class that does the *same* operations to every `band` (bias line).

In [ ]:
# configure a collection of bands as a single bias group.
grouped_biases = GroupedBiases(S=S, cfg=cfg, bias_group=bias_group, verbose=verbose)
grouped_biases.overbias_tes(sleep_time=120)

## The Optimization - AutoTune
AutoTune is a class that operates on an abstract number of `bands`. This class acquires time streams,
then makes a map of white noise levels with respect to band and up conversion attenuation settings.
An acceptance function is uses to determine if subsequent tuning is needed, tuning is repeated for 5 loops
or until every band exits the tuning successfully.

To operate on a single `band`, have the iterable `bias_group` contain only a single element,
for example, `bias_group=[4]`.

The idea of tuning with multiple biases at a time is to minimize the number of time streams needed to
fully tune and array.

In [ ]:
# acquire time stream data
auto_tune = AutoTune(S=S, cfg=cfg, nperseg=nperseg, bias_group=bias_group, verbose=verbose)
auto_tune.tune_selector_up_atten(uc_attens_centers_per_band=None, loop_count_max=5,
                                 stream_time=stream_time, do_plots=False, fmin=5, fmax=50)
# print the plotting directory
print(f"plotting directory is:\n{S.plot_dir}")